In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pyperclip
import time
import pandas as pd
import os
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException, StaleElementReferenceException




# 전역 변수 선언
UserName = ""  # 전역 변수로 초기화

# ✅ 브라우저 창 크기 설정
WINDOW_WIDTH = 1100
WINDOW_HEIGHT = 1000

# ✅ 브라우저 초기화 및 로그인 (기존 함수 유지)
def setup_browser(position_x):
    driver = webdriver.Chrome()
    driver.set_window_size(WINDOW_WIDTH, WINDOW_HEIGHT)
    driver.set_window_position(position_x, 0)
    driver.get('https://console.thebackend.io/ko/login')
    time.sleep(0.2)
    return driver

# ✅ 로그인 함수 (기존 함수 유지)
def login_to_console(driver):
    try:
        id_input = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "username")))
        pyperclip.copy("dksrufp0607@naver.com")
        id_input.send_keys(Keys.CONTROL, 'v')
        pw_input = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "password")))
        pyperclip.copy("ghkfkd159!")
        pw_input.send_keys(Keys.CONTROL, 'v')
        time.sleep(1)
        login_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[type="submit"]')))
        login_button.click()
        print("✅ 로그인 완료!")
        time.sleep(2)
    except Exception as e:
        print(f"⚠️ 로그인 실패: {e}")

# ✅ 콘솔 페이지 이동 (기존 함수 유지)
def navigate_to_page(driver):
    article_url = 'https://console.thebackend.io/ko/project/1ea3f14d34e89530ea88b3245bc82dc17d5f52ce1554049f19fce9219a847cfce18bb8891c9ffe90bc65e2b9a3b981853fc5513c1dd200afc9590ba6bfd5fced4230647d25328849e0917641/baseGameInfo/data'
    driver.get(article_url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    print("✅ 페이지 로드 완료!")

# ✅ "확인" 버튼 클릭 (기존 함수 유지)
def confirm_click(driver):
    try:
        print("🔍 '확인' 버튼 탐색 중...")
        confirm_button = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "확인")]')))
        confirm_button.click()
        print("✅ 확인 버튼 클릭 완료!")
    except TimeoutException:
        print("⚠️ '확인' 버튼을 찾지 못했습니다.")
    except Exception as e:
        print(f"⚠️ 확인 버튼 클릭 실패: {e}")

# ✅ CSV 데이터 로드 함수 (UserName 전역 변수 설정)
def get_csv_value(row, col):
    global UserName  # 전역 변수 UserName 사용
    global csv_path 
    try:
        df = pd.read_csv(csv_path)
        if row < df.shape[0] and col < df.shape[1]:
            print(f"🔹 {row}행 3열 값 (출력용): {df.iloc[row, 2]}")
            UserName = df.iloc[row, 2]  # UserName 설정
            return df.iloc[row, col]
        print("⚠️ CSV 파일의 크기를 초과하는 인덱스입니다.")
        return None
    except FileNotFoundError:
        print(f"⚠️ CSV 파일을 찾을 수 없습니다: {csv_path}")
        return None
    except Exception as e:
        print(f"⚠️ CSV 로드 중 오류 발생: {e}")
        return None

# ✅ 검색 실행 함수 (기존 함수 유지)
def execute_tasks(driver, row, col, type):
    try:
        
        b2_value = get_csv_value(row, col)
        if not b2_value:
            print("⚠️ CSV에서 값을 가져오지 못했습니다.")
            return
        table_dropdown = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//div[contains(@class, "_manageInfo-data_1rntr_71")]')))
        table_dropdown.click()
        print("✅ 테이블 드롭다운 클릭 완료!")
        base_data_option = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f'//div[@role="option"][span[text()="{type}"]]')))
        base_data_option.click()
        print("✅ BASE_DATA 선택 완료!")
        detail_search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//p[contains(text(), "상세 검색")]')))
        time.sleep(3)
        try:
            detail_search_button.click()
        except ElementClickInterceptedException:
            driver.execute_script("arguments[0].click();", detail_search_button)
            print("⚠️ '상세 검색' 버튼 클릭 차단됨, JavaScript로 실행")
        print("✅ 상세 검색 클릭 완료!")
        search_input = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.NAME, "defaultSearchValue")))
        search_input.clear()
        search_input.send_keys(b2_value)
        print(f"✅ B2 값 입력 완료: {b2_value}")
        search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "ui medium positive button") and @type="submit"]')))
        search_button.click()
        print("✅ 검색 버튼 클릭 완료!")
        uuid_element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//p[contains(@class, "_link_a1mor_113")]')))
        uuid_element.click()
        print("✅ UUID 클릭 완료!")
    except Exception as e:
        print(f"⚠️ 검색 작업 중 오류 발생: {e}")

# ✅ 데이터를 추출하는 함수 (기존 함수 유지)
def extract_data(driver):
    data = []
    try:
        # driver.switch_to.default_content()
        # driver.execute_script("document.body.style.zoom='25%'")
        # print("✅ 웹 화면 25% 축소 완료!")
        # time.sleep(2)
        context_found = False
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "_columnBorder_1rntr_40")))
            print("🔍 기본 콘텐츠에서 요소 발견!")
            context_found = True
        except TimeoutException:
            iframes = driver.find_elements(By.TAG_NAME, "iframe")
            for i, frame in enumerate(iframes):
                driver.switch_to.frame(frame)
                try:
                    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "_columnBorder_1rntr_40")))
                    print(f"🔍 iframe {i}에서 요소 발견!")
                    context_found = True
                    break
                except TimeoutException:
                    driver.switch_to.default_content()
            if not context_found:
                print("⚠️ 모든 컨텍스트에서 요소를 찾지 못함")
                return data
        elements = WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "_columnBorder_1rntr_40")))
        print(f"✅ 찾은 요소 개수: {len(elements)}")
        for idx, element in enumerate(elements):
            try:
                column_name = element.find_element(By.CSS_SELECTOR, "div.inline.field:nth-child(1) p").text
                data_type = element.find_element(By.CSS_SELECTOR, "div.inline.field:nth-child(2) p").text
                value = "N/A"
                print(f"🔍 처리 중: {column_name} (타입: {data_type})")
                driver.execute_script("arguments[0].scrollIntoView(true);", element)
                time.sleep(0.5)
                try:
                    ace_editor = element.find_element(By.CSS_SELECTOR, "div.ace_editor")
                    driver.execute_script("arguments[0].scrollIntoView(true);", ace_editor)
                    WebDriverWait(driver, 10).until(
                        lambda d: d.execute_script("return typeof ace !== 'undefined' && ace.edit(arguments[0]).session.getValue().length > 0", ace_editor)
                    )
                    value = driver.execute_script("return ace.edit(arguments[0]).getValue();", ace_editor).strip()
                    print(f"📋 Ace Editor에서 값 추출: {value}")
                except NoSuchElementException:
                    alternative_selectors = [
                        ("div.inline.field:nth-child(3) p", "일반 텍스트"),
                        ("textarea", "텍스트 영역"),
                        ("pre", "Preformatted 텍스트"),
                        ("div[class*='value']", "값을 포함한 div")
                    ]
                    for selector, desc in alternative_selectors:
                        try:
                            value_elem = element.find_element(By.CSS_SELECTOR, selector)
                            value = value_elem.text.strip()
                            print(f"📋 {desc}에서 값 추출: {value}")
                            break
                        except NoSuchElementException:
                            continue
                    if value == "N/A":
                        print(f"⚠️ {column_name}에서 값 추출 실패, HTML: {element.get_attribute('outerHTML')[:200]}...")
                data.append([column_name, data_type, value])
                print(f"✅ {column_name} 처리 완료: {value}")
            except NoSuchElementException:
                print(f"⚠️ 요소 {idx+1}에서 컬럼 이름/타입 찾기 실패")
            except Exception as e:
                print(f"⚠️ 요소 {idx+1} 처리 중 오류: {e}")
    except Exception as e:
        print(f"⚠️ 데이터 추출 실패: {e}")
    finally:
        driver.switch_to.default_content()
    return data

# ✅ 엑셀 저장 함수 (기존 함수 유지)
def save_to_excel(data, filename):
    try:
        df = pd.DataFrame(data, columns=["Column Name", "Data Type", "Value"])
        df.to_excel(filename, index=False, engine="openpyxl")
        print(f"✅ 데이터가 {os.path.abspath(filename)}에 저장되었습니다.")
    except Exception as e:
        print(f"⚠️ 엑셀 저장 실패: {e}")




# Ranking 배열 정의 (예: 1부터 5까지)
Ranking = [1,2]  # 원하는 랭킹 값들로 수정 가능

csv_path = "C:/엑셀/Ranking_1.csv"
type = ["TREASURE_DATA", "GROWTH3_DATA", "BASE_DATA"]
# Ranking = [i for i in range(1)] # 원하는 랭킹까지 복사
size = len(Ranking)        

# ✅ 메인 실행 (Ranking 배열로 루프 추가)
if __name__ == "__main__":

    


    

        print(f"🔍 현재 작업 디렉토리: {os.getcwd()}")

        driver1 = setup_browser(0)
        try:
            login_to_console(driver1)
            # Ranking 배열을 루프로 순회
            for rank in Ranking:
            
                
                for i in range(3):
                    navigate_to_page(driver1)
                    confirm_click(driver1)
                    print(f"\n🔄 랭킹 {rank} 처리 시작")
                    execute_tasks(driver1, rank - 1, 1 , type[i])  # rank-1은 CSV 행 인덱스 조정
                    extracted_data = extract_data(driver1)
                    if extracted_data:
                        excel_filename = f"C:\\엑셀\\복사\\{type[i]}\\{UserName}.xlsx"  # UserName 기반 파일명
                        save_to_excel(extracted_data, excel_filename)
                    else:
                        print(f"⚠️ 랭킹 {rank}에서 추출된 데이터가 없습니다.")
                    time.sleep(2)  # 각 루프 사이 안정화 대기

        except KeyboardInterrupt:
            print("⚠️ 사용자가 프로그램을 중단했습니다.")
        except Exception as e:
            print(f"⚠️ 전체 실행 중 오류: {e}")
        finally:
            print("✅ 브라우저가 종료되었습니다.")
            # driver1.quit()  # 주석 해제 필요 시

🔍 현재 작업 디렉토리: c:\JJG\Start\sjdj
✅ 로그인 완료!
✅ 페이지 로드 완료!
🔍 '확인' 버튼 탐색 중...
⚠️ '확인' 버튼을 찾지 못했습니다.

🔄 랭킹 1 처리 시작
🔹 0행 3열 값 (출력용): 1
✅ 테이블 드롭다운 클릭 완료!
✅ BASE_DATA 선택 완료!
✅ 상세 검색 클릭 완료!
✅ B2 값 입력 완료: a0d328f0-0549-11f0-aec8-61abfd6c7ecd
✅ 검색 버튼 클릭 완료!
✅ UUID 클릭 완료!
🔍 기본 콘텐츠에서 요소 발견!
✅ 찾은 요소 개수: 3
🔍 처리 중: hero_treasureBaseSeed (타입: object)
📋 Ace Editor에서 값 추출: [
    53026,
    46744,
    52108,
    7035,
    32537,
    73009,
    76111,
    33024,
    77870,
    82476,
    94895,
    36303,
    1619,
    13276,
    97376,
    75303,
    51744,
    63788,
    41827,
    38982,
    85239,
    88085,
    84122,
    46910,
    61583,
    3976,
    9935,
    55291,
    47065,
    1870,
    28515,
    78863,
    58469,
    24908,
    16894,
    9510,
    23824,
    22658,
    11675,
    467,
    66548,
    82492,
    66637,
    67667,
    24278,
    12122,
    948,
    69875,
    41266,
    78006,
    74212,
    54664,
    6131,
    41333,
    64159,
    36363,
    72747,
    30661
]
✅ hero_treasur